In [1]:
import pandas as pd
from pathlib import Path

In [11]:
CP = [
    "..\\results\Assn2_Q5\ClusterProfiler\CP_enrichment_results_t0_t24.csv",
    "..\\results\Assn2_Q5\ClusterProfiler\CP_enrichment_results_t0_t48.csv",
    "..\\results\Assn2_Q5\ClusterProfiler\CP_enrichment_results_t0_t72.csv",
    "..\\results\Assn2_Q5\ClusterProfiler\CP_enrichment_results_t24_t48.csv",
    "..\\results\Assn2_Q5\ClusterProfiler\CP_enrichment_results_t24_t72.csv",
    "..\\results\Assn2_Q5\ClusterProfiler\CP_enrichment_results_t48_t72.csv",
]
GP = [
    "..\\results\Assn2_Q5\GProfiler2\GProf_enrichment_results_t0_t24.csv",
    "..\\results\Assn2_Q5\GProfiler2\GProf_enrichment_results_t0_t48.csv",
    "..\\results\Assn2_Q5\GProfiler2\GProf_enrichment_results_t0_t72.csv",
    "..\\results\Assn2_Q5\GProfiler2\GProf_enrichment_results_t24_t48.csv",
    "..\\results\Assn2_Q5\GProfiler2\GProf_enrichment_results_t24_t72.csv",
    "..\\results\Assn2_Q5\GProfiler2\GProf_enrichment_results_t48_t72.csv",
]
GO = [
    "..\\results\Assn2_Q5\\topGO\coupled\GO_enrichment_results_t0_t24.csv",
    "..\\results\Assn2_Q5\\topGO\coupled\GO_enrichment_results_t0_t48.csv",
    "..\\results\Assn2_Q5\\topGO\coupled\GO_enrichment_results_t0_t72.csv",
    "..\\results\Assn2_Q5\\topGO\coupled\GO_enrichment_results_t24_t48.csv",
    "..\\results\Assn2_Q5\\topGO\coupled\GO_enrichment_results_t24_t72.csv",
    "..\\results\Assn2_Q5\\topGO\coupled\GO_enrichment_results_t48_t72.csv"
]

In [12]:
CP_KEY = "ID"
GP_KEY = "term_id"
GO_KEY = "GO.ID"

In [16]:
for i, (cp, gp, go) in enumerate(zip(CP, GP, GO), start=1):
    dcp = pd.read_csv(Path(cp).resolve().absolute())
    dgp = pd.read_csv(Path(gp).resolve().absolute())
    dgo = pd.read_csv(Path(go).resolve().absolute())

    # Standardize key column name across all dataframes
    dcp = dcp.rename(columns={CP_KEY: "key"})
    dgp = dgp.rename(columns={GP_KEY: "key"})
    dgo = dgo.rename(columns={GO_KEY: "key"})

    # Add source tag to avoid column name collisions
    dcp = dcp.add_prefix("cp_")
    dgp = dgp.add_prefix("gp_")
    dgo = dgo.add_prefix("go_")

    # Restore key column name
    dcp = dcp.rename(columns={"cp_key": "key"})
    dgp = dgp.rename(columns={"gp_key": "key"})
    dgo = dgo.rename(columns={"go_key": "key"})

    # Merge all on the "key" column (outer join to include all keys)
    merged = pd.merge(dcp, dgp, on="key", how="outer")
    merged = pd.merge(merged, dgo, on="key", how="outer")

    # Count number of appearances of each key across the 3 files
    merged["count"] = (
        merged["key"].isin(dcp["key"]).astype(int)
        + merged["key"].isin(dgp["key"]).astype(int)
        + merged["key"].isin(dgo["key"]).astype(int)
    )

    cols = ["key", "count"] + [c for c in merged.columns if c not in ("key", "count")]
    merged = merged[cols]

    merged = merged.sort_values(by="count", ascending=False)

    # Save output CSV
    out_file = f"combined_{i}.csv"
    merged.to_csv(out_file, index=False)
    print(f"Saved {out_file}")

Saved combined_1.csv
Saved combined_2.csv
Saved combined_3.csv
Saved combined_4.csv
Saved combined_5.csv
Saved combined_6.csv
